# Homework 1 - LP duality and Revenue Management

In [152]:
import numpy as np
from gurobipy import *

## 1 Semiconductor Manufacturing

### Part 1: LP Formulation

In [153]:
# Create the model. 
m = Model()

# Add the variables. 
# addVar() will just add a single variable.
x_1 = m.addVar() #this creates a continuous variable with a minmun of 0 and maximun of infinity, no need for non-negativity
x_2 = m.addVar()
x_3 = m.addVar()

# Create the constraints. 
# Resource constraints:
Ge_transistors = m.addConstr( 14*x_1 + 20*x_2 +  40*x_3  <= 300)
Si_transistors=  m.addConstr(30*x_1 + 20*x_2 +  15*x_3  <= 200)
Time_minutes =  m.addConstr(  20*x_1 + 30*x_2 + 50*x_3  <= 1080)

# Create the objective function.
m.setObjective(500*x_1 + 800*x_2 + 1000*x_3, GRB.MAXIMIZE)

# Update and solve
m.update()

m.optimize()

###############################################
# Print Optimal Solution
print("Optimal Value:", m.objVal)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 3 columns and 9 nonzeros
Model fingerprint: 0xc6454fc5
Coefficient statistics:
  Matrix range     [1e+01, 5e+01]
  Objective range  [5e+02, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 1e+03]
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 3 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2000000e+04   8.114000e+00   0.000000e+00      0s
       2    9.6000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds (0.00 work units)
Optimal objective  9.600000000e+03
Optimal Value: 9600.0


In [154]:
# Extract the solution status. 
status = m.status
print(status)
if status == GRB.OPTIMAL:
    print("Solved to optimality")
    
# Extract the optimal values of the decision variables.
Chip_1_value = x_1.x
Chip_2_value = x_2.x
Chip_3_value = x_3.x

print("Number of Chips Type 1: ", Chip_1_value)
print("Number of Chips Type 2: ", Chip_2_value)
print("Number of Chips Type 3: ", Chip_3_value)

# Extract the optimal objective value.
optimal_obj_original = m.objval
print("Optimal objective: ", optimal_obj_original)

2
Solved to optimality
Number of Chips Type 1:  0.0
Number of Chips Type 2:  7.0
Number of Chips Type 3:  4.0
Optimal objective:  9600.0


### Part 2: Dual Formulation

In [155]:
m_dual = Model()

Ge = m_dual.addVar()
Si = m_dual.addVar()
Minutes = m_dual.addVar()

# Create the dual constraints
m_dual.addConstr( 14*Ge + 30*Si + 20*Minutes >= 500 )
m_dual.addConstr( 20*Ge + 20*Si + 30*Minutes  >= 800)
m_dual.addConstr( 40*Ge + 15*Si + 50*Minutes  >= 1000 )

m_dual.setObjective( 300 * Ge + 200 * Si + 1080 * Minutes, GRB.MINIMIZE)

m_dual.update()

m_dual.optimize()


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 3 columns and 9 nonzeros
Model fingerprint: 0xcdc0bb64
Coefficient statistics:
  Matrix range     [1e+01, 5e+01]
  Objective range  [2e+02, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+02, 1e+03]
Presolve removed 0 rows and 1 columns
Presolve time: 0.01s
Presolved: 3 rows, 2 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.875000e+02   0.000000e+00      0s
       2    9.6000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds (0.00 work units)
Optimal objective  9.600000000e+03


In [156]:
Ge_value = Ge.x
Si_value = Si.x
Minutes_value = Minutes.x

print("Dual variable for Ge Transistors: ", Ge_value)
print("Dual variable for Si Transistors: ", Si_value)
print("Dual variable for Minutes: ", Minutes_value)

# Extract the dual optimal objective value.
dual_optimal_obj = m_dual.objval
print("Dual optimal objective: ", dual_optimal_obj)

Dual variable for Ge Transistors:  16.0
Dual variable for Si Transistors:  24.0
Dual variable for Minutes:  0.0
Dual optimal objective:  9600.0


Verifying that optimal values of dual variables are equal to the shadow prices of the formulation in part 1:

In [157]:
## Shadow prices of the constraints part 1
print(f"Shadow price of Ge transistor: {Ge_transistors.pi}")
print(f"Shadow price of Si transistor: {Si_transistors.pi}")
print(f"Shadow price of time minutes: {Time_minutes.pi}")

Shadow price of Ge transistor: 16.0
Shadow price of Si transistor: 24.0
Shadow price of time minutes: 0.0


### Part 3: Shadow Prices / Marginal Analysis

**a):** No, increasing the amount of manufacturing time by 10 hours is not going to improve our revenue. This is because as we saw in our optimal dual variable value for the constraint of minutes, adding an additional small change in minutes at the RHS will return 0.0 improvement in our revenue. Each additional minute of manufacturing process would bring an additional marginal revenue of $USD 0. Given that the shadow price of minutes is 0, it means that the constraint is not binding. This implies that the optimal solution will not change if the constraint is relaxed. This is because the constraint is not limiting the optimal solution.

**b):** Given that the shadow price of Ge transistor is 16, an increase in 1 billion Ge transistors will yield USD 16 aditional revenue. Therefore, I expect that an increase of 10 billion Ge transistors will yield $USD 160 of additional revenue. A total of 9600 + 160 = 9760

**c):** When I solved the optimization problem in part 1 by modifying the RHS of the constraint, I could verify the prediction in part 3 b. 

In [158]:
Ge_transistors.rhs = 310
m.update()
m.optimize()
optimal_obj_change_1 = m.objval
print("Optimal objective: $USD ", optimal_obj_change_1)
print("Increase in revenue by increasing Ge by 10: $USD ", optimal_obj_change_1 - optimal_obj_original)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 3 columns and 9 nonzeros
Coefficient statistics:
  Matrix range     [1e+01, 5e+01]
  Objective range  [5e+02, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 1e+03]
LP warm-start: use basis
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.7600000e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  9.760000000e+03
Optimal objective: $USD  9760.0
Increase in revenue by increasing Ge by 10: $USD  160.0


**d):** In this case, since we are duplicating the amount of Ge transistors (It is not a small change), I believe that the result **is not** going to be the estimation 300*Ge_shadow_price = $USD 4800 increase in revenue. The increase in revenue should be less than $USD 4800. This is because I expect that either the constraint minutes or Si transistors are now going to be limiting the optimal solution.

**e):**

In [159]:
Ge_transistors.rhs = 600
m.update()
m.optimize()
optimal_obj_change_2 = m.objval
print(f"Optimal objective: $USD {optimal_obj_change_2:.0f}")
print(f"Increase in revenue by increasing Ge by 300: $USD {(optimal_obj_change_2 - optimal_obj_original):.0f}")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 3 columns and 9 nonzeros
Coefficient statistics:
  Matrix range     [1e+01, 5e+01]
  Objective range  [5e+02, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 1e+03]
LP warm-start: use basis
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4400000e+04   8.000000e+00   0.000000e+00      0s
       1    1.3333333e+04   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.333333333e+04
Optimal objective: $USD 13333
Increase in revenue by increasing Ge by 300: $USD 3733


**Ans:** There are discrepancies because as I mentioned before, the constraint minutes, Si or both are going to be binding the optimal solution. That is, limiting the production of chips and therefore the revenue. 

**f):** in this case, since we are changing more than one constraint, and the changes are small perturbations of $\delta$, then the approximation will hold. Using shadow prices, I can expect the increase in revenue to be: 16\*10 + 24\*20 = $USD 640

**g)** When increasing Ge by 10 and Si by 20, and modifying the rhs of the constraints Ge_transistors to 310 and Si_transistors to 220. We can verify that the new optimal revenue expected is USD 10240. And the increase in comparison to part 1-b) is 640 as stated before. 

In [160]:
Ge_transistors.rhs = 310
Si_transistors.rhs = 220
m.update()
m.optimize()
optimal_obj_change_3 = m.objval
print(f"Optimal objective: $USD {optimal_obj_change_3:.0f}")
print(f"Increase in revenue by increasing Ge by 10 and Si by 20: $USD {(optimal_obj_change_3 - optimal_obj_original):.0f}")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 3 columns and 9 nonzeros
Coefficient statistics:
  Matrix range     [1e+01, 5e+01]
  Objective range  [5e+02, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 1e+03]
LP warm-start: use basis
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4666667e+04   3.458333e+01   0.000000e+00      0s
       1    1.0240000e+04   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.024000000e+04
Optimal objective: $USD 10240
Increase in revenue by increasing Ge by 10 and Si by 20: $USD 640


## 2 - Cloud Computing

### Part 2: Solving Capacity control problem

In [161]:
# The number of instances
nInstances = 9

# price for each instance type:
price = np.array([7, 12, 24, 22, 44, 88, 30, 90, 120])

# rate of each instance type: 
rate = np.array([5.0, 5.0, 1.8, 3.0, 2.6, 1.0, 0.8, 0.4, 0.3])

# Time horizon:
T = 5

# Finally, let us compute the forecasted demand for each fare type:
forecast = T * rate

# Parametrers
CPU = np.array([16, 32, 64, 8, 16, 32, 16, 32, 64])
Memory = np.array([8, 16, 32, 32, 64, 128, 16, 32, 64])
GPU = np.array([1, 1, 1, 1, 1, 1, 2, 6, 8])

m = Model()

# Add the decision variable
x = m.addVars(nInstances, lb = 0, ub = forecast)


# Create the seat constraint
CPU_constr = m.addConstr( sum(x[i] * CPU[i] for i in range(nInstances)) <= 512 )
Memory_constr = m.addConstr( sum(x[i] * Memory[i] for i in range(nInstances)) <= 1024 )
GPU_constr = m.addConstr( sum(x[i] * GPU[i] for i in range(nInstances)) <= 64 )

for i in range(nInstances):
    m.addConstr(x[i] >= 0)


for i in range(nInstances):
    m.addConstr( x[i] <= forecast[i] )

# Set the objective.
m.setObjective(sum( price[i] * x[i] for i in range(nInstances)), GRB.MAXIMIZE )

# Update + solve:
m.update()
m.optimize()

# Get the objective value
LP_obj = m.objval

# Get the allocation
allocation = np.array([x[i].x for i in range(nInstances)])

# Display the results:
print("Allocation, forecast, objective:")
print(allocation)
print(forecast)
print(LP_obj)


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 21 rows, 9 columns and 45 nonzeros
Model fingerprint: 0xae8ece77
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [7e+00, 1e+02]
  Bounds range     [2e+00, 3e+01]
  RHS range        [2e+00, 1e+03]
Presolve removed 18 rows and 1 columns
Presolve time: 0.01s
Presolved: 3 rows, 8 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4400000e+03   7.200000e+01   0.000000e+00      0s
       3    1.0394286e+03   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.039428571e+03
Allocation, forecast, objective:
[6.28571429 0.         0.         3.42857143 0.         5.
 4.         2.         1.5

### Part 3: simulating current practice

In [162]:
# Preconditions:
# nSimulations = integer specifying number of simulations to run
# rates = array containing arrival rate (# / day) for each of the instance
# types (should be an array with 9 elements)
# T = length of horizon in days.

def generateArrivalSequences( nSimulations, rates, T ):
    total_rate = sum(rates)
    nTypes = len(rates)

    arrival_sequences_times = []
    arrival_sequences_types = [];

    for s in range(nSimulations):
        single_arrival_sequence_time = [];
        single_arrival_sequence_type = [];
        t = 0;
        while (t < T):
            single_time = np.random.exponential(1.0/total_rate)
            single_type = np.random.choice(nTypes, p= rates/total_rate )

            t += single_time;

            if (t < T):
                single_arrival_sequence_time.append(t)
                single_arrival_sequence_type.append(single_type)
            else:
                break

        arrival_sequences_times.append(np.array(single_arrival_sequence_time))
        arrival_sequences_types.append(np.array(single_arrival_sequence_type))
    return arrival_sequences_times, arrival_sequences_types



# Code to test out above function
np.random.seed(10)
nSimulations_test = 100
rates_test = np.array([5.0, 5.0, 1.8, 3.0, 2.6, 1.0, 0.8, 0.4, 0.3])
T_test = 5
x, y = generateArrivalSequences(nSimulations_test, rates_test, T_test)

print(f"the first three times in arrival sequences times[0] should be: {x[0][:3]}")

C1_count = sum(np.count_nonzero(y[i] == 0) for i in range(nSimulations_test))
print(f"Average number of arrivals of type C1 in the set of simulated sequences: {C1_count / 100}")



the first three times in arrival sequences times[0] should be: [0.07414243 0.1246028  0.15928449]
Average number of arrivals of type C1 in the set of simulated sequences: 26.63


**b):**

In [163]:
total_number_requests = sum( y[i].size for i in range(nSimulations_test))
print(f"Average number of arrivals of all types in the set of simulated sequences: {total_number_requests / 100}")

Poisson_total = sum(T * rate)
print(f"Expected value of the sum of Poisson random variables: {Poisson_total}")

Average number of arrivals of all types in the set of simulated sequences: 101.0
Expected value of the sum of Poisson random variables: 99.5


In [164]:
# Preconditions for code below:
# nSimulations = number of simulations to run
# nResources = number of different types of resources (= 3)
# B = numpy array of initial capacities of the resources
# arrival_sequences_times = array where each entry is arrival time sequence for that
# simulation
# arrival_sequences_types = array where each entry is sequence of request types for
# that simulation

nSimulations = 100
nResources = 3

B = np.array([512, 1024, 64])
price = np.array([7, 12, 24, 22, 44, 88, 30, 90, 120])

arrival_sequences_times = x
arrival_sequences_types = y

# Parametrers
CPU = np.array([16, 32, 64, 8, 16, 32, 16, 32, 64])
Memory = np.array([8, 16, 32, 32, 64, 128, 16, 32, 64])
GPU = np.array([1, 1, 1, 1, 1, 1, 2, 6, 8])

# Note: code will not run; parts with ... need to be filled in.

results_myopic_revenue = np.zeros(nSimulations)
results_myopic_remaining_capacity = np.zeros((nResources, nSimulations))

for s in range(nSimulations):
    b = B.copy()
    single_revenue = 0.0 # will contain the revenue of this simulation
    nArrivals = len(arrival_sequences_times[s])

    # Go through the arrivals in sequence
    for j in range(nArrivals):
        # Obtain the time of the arrival, and its type (i)
        arrival_time = arrival_sequences_times[s][j]
        i = arrival_sequences_types[s][j]

        # Check if there is sufficient capacity for the request
        if ( CPU[i] <= b[0] and Memory[i] <= b[1] and GPU[i] <= b[2] ):
            # If there is sufficient capacity, accrue the revenue
            single_revenue += price[i]
            # and remove the capacity.
            b[0] -= CPU[i]
            b[1] -= Memory[i]
            b[2] -= GPU[i] 

    # Save the results of this simulation here
    results_myopic_revenue[s] = single_revenue
    results_myopic_remaining_capacity[:, s] = b

# Find the average revenue
average_revenue = np.mean(results_myopic_revenue)
print(f"Average revenue: {average_revenue}")

# Find the average remaining quantity of each resource
average_remaining_capacity = np.mean(results_myopic_remaining_capacity, axis = 1)
print(f"Average remaining capacity: {average_remaining_capacity}")


Average revenue: 528.28
Average remaining capacity: [2.4000e-01 3.7152e+02 3.7420e+01]


### Part 4: A bid-price control policy

In [165]:
# Preconditions for code below:
# nSimulations = number of simulations to run
# nResources = number of different types of resources (= 3)
# B = numpy array of initial capacities of the resources
# arrival_sequences_times = array where each entry is arrival time sequence for that
# simulation
# arrival_sequences_types = array where each entry is sequence of request types for
# that simulation
nInstances = 9
nSimulations = 100
nResources = 3

B = np.array([512, 1024, 64])
price = np.array([7, 12, 24, 22, 44, 88, 30, 90, 120])

# arrival rate per day
rate = np.array([5.0, 5.0, 1.8, 3.0, 2.6, 1.0, 0.8, 0.4, 0.3])

# Time horizon:
T = 5

# Finally, let us compute the forecasted demand for each instance type:
forecast = T * rate

# Parametrers
CPU = np.array([16, 32, 64, 8, 16, 32, 16, 32, 64])
Memory = np.array([8, 16, 32, 32, 64, 128, 16, 32, 64])
GPU = np.array([1, 1, 1, 1, 1, 1, 2, 6, 8])

# Your LP formulation should be defined here
# (or somewhere before the definition of bpc()...)
m_bid = Model()

# Add the decision variable
x = m_bid.addVars(nInstances, lb = 0, ub = forecast)

# Create the capacity constraint
CPU_constr = m_bid.addConstr( sum(x[i] * CPU[i] for i in range(nInstances)) <= 512 )
Memory_constr = m_bid.addConstr( sum(x[i] * Memory[i] for i in range(nInstances)) <= 1024 )
GPU_constr = m_bid.addConstr( sum(x[i] * GPU[i] for i in range(nInstances)) <= 64 )

# Set the objective.
m_bid.setObjective(sum( price[i] * x[i] for i in range(nInstances)), GRB.MAXIMIZE )

m_bid.setParam('OutputFlag', False )

# As we did in-class, define a function bpc() to re-solve the LO each time:
def bpc(b, t):
    dual_val = np.zeros(nResources)
    for r in range(nResources):
        # Set the RHS of the resource constraint to b[r] here
        if r == 0:
            CPU_constr.rhs = b[r]
        elif r == 1:
            Memory_constr.rhs = b[r]
        elif r == 2:
            GPU_constr.rhs = b[r]

    for i in range(nInstances):
        # Set the RHS of the forecast constraint for each instance
        # type to the expected number of requests over the duration
        # of the remaining horizon (T - t).
        x[i].ub = (T - t)*rate[i]
        # ...

    # Re-solve the model:
    m_bid.update()
    m_bid.optimize()

    # Obtain the dual values/shadow prices
    dual_val[0] = CPU_constr.pi
    dual_val[1] = Memory_constr.pi
    dual_val[2] = GPU_constr.pi 

    # Return the dual values:
    return dual_val

results_revenue = np.zeros(nSimulations)
results_remaining_capacity = np.zeros((nResources, nSimulations))

for s in range(nSimulations):
    b = B.copy() #Initialize the current capacity
    single_revenue = 0.0 #Initialize the revenue garnered in this simulation
    nArrivals = len(arrival_sequences_times[s])
    
    for j in range(nArrivals):
        # Take the next arrival time and type from the sequence
        arrival_time = arrival_sequences_times[s][j]
        i = arrival_sequences_types[s][j]

        if (CPU[i] <= b[0] and Memory[i] <= b[1] and GPU[i] <= b[2]):
            # Re-solve the LO and obtain the dual values
            dual_val = bpc(b, arrival_time)
            # Compute the total bid price of the request
            total_bid_price = sum(dual_val * np.array([CPU[i], Memory[i], GPU[i]]))

            # Check if the revenue is at least the sum of the bid prices:
            if ( price[i] >= total_bid_price ):
                # If there is sufficient capacity, accrue the revenue
                single_revenue += price[i]
                # and remove the capacity.
                b[0] -= CPU[i]
                b[1] -= Memory[i]
                b[2] -= GPU[i] 

    # Save the results of this simulation here:
    results_revenue[s] = single_revenue
    results_remaining_capacity[:, s] = b

# Find the average revenue
average_revenue = np.mean(results_revenue)
print(f"Average revenue: {average_revenue}")

# Find the average remaining quantity of each resource
average_remaining_capacity = np.mean(results_remaining_capacity, axis = 1)
print(f"Average remaining capacity: {average_remaining_capacity}")

Average revenue: 925.59
Average remaining capacity: [27.2   4.88 20.62]
